In [1]:
from generate_data import *
from data_imports import *
from new_data_structures import *
from caption_model import *

C:\Program Files\Anaconda3\envs\pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
cap_to_img = {12: 123,
              31: 123,
              21: 321,
              13: 231}
imgs = [123, 321, 231]
K = 2

In [3]:
generate_good_bad(cap_to_img, imgs, K)

array([[ 12, 123, 123],
       [ 12, 123, 231],
       [ 31, 123, 123],
       [ 31, 123, 123],
       [ 21, 321, 231],
       [ 21, 321, 231],
       [ 13, 231, 231],
       [ 13, 231, 321]])

In [4]:
d = load_json()

In [5]:
d.keys()

dict_keys(['info', 'images', 'licenses', 'annotations'])

In [6]:
stops = load_stopwords()

In [7]:
glove = load_embeddings()

elapsed 25.65948224067688s


In [8]:
resnet = load_resnet()

In [9]:
id_to_string = create_caption_id_to_string(d["annotations"], resnet)

In [10]:
import random
id_to_string_reduced = {k:v for k,v in id_to_string.items() if random.random() < 0.01}

In [11]:
# extremely slow - had to reduce
caps = reduce_captions(id_to_string_reduced, glove, stops)

In [12]:
new_captions = create_caption_id_to_embedding_string_img_id(d["annotations"], caps, id_to_string)

In [13]:
new_captions.keys()

dict_keys(['img_id', 'semantics', 'string'])

In [14]:
new_images = create_image_to_urls_resnet(d["images"], resnet)

In [15]:
new_images.keys()

dict_keys(['urls', 'resnet'])

In [16]:
triples = generate_good_bad(new_captions["img_id"], list(new_images["urls"].keys()), 3)

In [17]:
cap_ids = triples[:, 0]
good_ids = triples[:, 1]
bad_ids = triples[:, 2]

In [18]:
# not fast but whatever
#cap = np.stack([new_captions["semantics"][cap_id] for cap_id in cap_ids], axis=0) # breaks because used reduced input
good = np.concatenate([new_images["resnet"][img_id] for img_id in good_ids], axis=0)
bad = np.concatenate([new_images["resnet"][img_id] for img_id in bad_ids], axis=0)

In [19]:
good.shape

(1239774, 512)

Take **cap**, **good**, and **bad** as inputs to the training cycle

Do the following at the beginning of every epoch:

In [20]:
good[0][:5]

array([0.66306174, 0.72584295, 2.502352  , 3.057971  , 1.180764  ],
      dtype=float32)

In [27]:
import sklearn
good, bad = sklearn.utils.shuffle(good, bad); # add cap here once it works

In [22]:
good[0][:5]

array([0.3544872 , 0.6698576 , 2.191376  , 0.58562005, 0.9054055 ],
      dtype=float32)